In [1]:
from seleniumbase import Driver
from seleniumbase import page_actions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import os
import requests
import time
import csv
import spacy
import en_core_web_sm

In [2]:
class Skill:
    def __init__(self,name,dir, keyword , groups=None,prerequisites= None):
        filename = name
        filename = filename.replace('/','-')
        filename = filename.replace("\\",'-')
        filename = filename + ".html"
        path = os.path.join(dir, filename)
        path = path.replace("\\",'/')
        self.resource_path = path # for the resource path
        self.keyword_search = keyword  # keyword for searching LLM
        self.group_set = set()
        if groups is not None:    
            self.UpdateGroupSet(groups)
        self.prerequisite  = set()
        if prerequisites is not None:
            self.UpdatePrerequisites(prerequisites)
        

    def UpdateGroupSet(self,groups):
        self.group_set.update(groups)
        #print("skill group set updated.")
      
     
    def UpdatePrerequisites(self,prerequisites):
        self.prerequisite.update(prerequisites)
        #print("prerequisite updated.")
        
class Group:
    def __init__(self,name,skills):
        filename = name
        filename = filename.replace('/','-')
        filename = filename.replace("\\",'-')
        filename = filename + ".html"
        path = os.path.join("group", filename)
        path = path.replace("\\",'/')
        self.resource_path = path # for the resource path
        self.keyword_search = name + " technology" # keyword for searching LLM
        self.skill_set = skills

    def UpdateSkillSet(self,skill):
        self.skill_set.update(skill)
        #print("group skill set updated.")

    def ChangeKeyword(self,keyword):
        self.keyword_search = keyword

class Role:
    def __init__(self,name,skills,groups):
        filename = name
        filename = filename.replace('/','-')
        filename = filename.replace("\\",'-')
        filename = filename + ".html"
        path = os.path.join("role", filename)
        path = path.replace("\\",'/')
        self.resource_path = path # for the resource path
        self.keyword_search = name + " role" # keyword for searching LLM
        self.skill_set = set()
        if skills is not None:    
            self.UpdateSkillSet(skills)
        self.group_set = set()
        if groups is not None:    
            self.UpdateGroupSet(groups)

    def UpdateSkillSet(self,skills):
        if skills is not None:    
            self.skill_set.update(skills)
            #print("group skill set updated.")

    def UpdateGroupSet(self,groups):
        if groups is not None:   
            self.group_set.update(groups)
            #print("skill group set updated.")

    def ChangeKeyword(self,keyword):
        self.keyword_search = keyword
    
    
        

In [3]:
class TechStack:
    def __init__(self):
        self.skill_dict_list = {}
        self.group_dict_list = {}
        self.alias_dict_list = {}
        self.ignore_set = set()
        self.not_found_set = set()
        self.role_dict_list = {}

               
    def AddSkillDictList(self,name,path,keyword,groups=None,prerequisites = None):
        if name in self.alias_dict_list:
            name = self.alias_dict_list[name]
            
        if name not in self.skill_dict_list:
            self.skill_dict_list[name] = Skill(name,path,keyword,groups,prerequisites)
            #print(name,"added in skill_dict_list.")
            if groups is not None:
                for g in groups:
                    if g in self.group_dict_list:
                        self.group_dict_list.get(g).UpdateSkillSet({name})
                        #print(name,"added in",g,".")
                    else:
                        self.group_dict_list[g] = Group(g,{name})
                        #print("new group:",g,"have been created and added",name,".")
        else:
            self.UpdateSkillDictList(name,groups,prerequisites)
                    
    def UpdateSkillDictList(self,name,groups,prerequisites):
        if name in self.skill_dict_list:
            self.skill_dict_list[name].UpdateGroupSet(groups)
            if prerequisites is not None:
                self.skill_dict_list[name].UpdatePrerequisites(prerequisites)
                #print(name,"prerequisites updated.")

    def AddGroupDictList(self,name,skills):
        if skills is not None:
            if name in self.group_dict_list:
                self.UpdateGroupDictList(name,skills)
            else:
                found_set = set()
                for s in skills:
                    if s in self.skill_dict_list:
                        self.skill_dict_list[s].UpdateGroupSet({name})
                        found_set.add(s)  
                        #print(s,"added in",name,"group set.")
                self.group_dict_list[name] = Group(name,found_set)

    def UpdateGroupDictList(self,name,skills):
        if name in self.group_dict_list:
            found_set = set()
            for s in skills:
                if s in self.skill_dict_list:
                      found_set.add(s)  
            self.group_dict_list[name].UpdateSkillSet(found_set)
        else:
            self.AddGroupDictList(name,skills)

    def AddAliasDictList(self,key,value):
        if key not in self.alias_dict_list:
            self.alias_dict_list[key]=value
            #print(key,"alias as",value,"created.")
            
    def AddIgnoreList(self,name):
        self.ignore_set.add(name)

    def AddRoleDictList(self,name,skills,groups):
        if name in self.role_dict_list:
            if skills is not None:
                skill_found_set = set()
                for s in skills:
                    if s in self.skill_dict_list:
                        skill_found_set.add(s)  
                self.role_dict_list[name].UpdateSkillSet(skill_found_set)
            if groups is not None:
                group_found_set = set()
                for g in groups:
                    if g in self.group_dict_list:
                        group_found_set.add(g)  
                self.role_dict_list[name].UpdateGroupSet(group_found_set)
        else:
            self.role_dict_list[name] = Role(name,skills,groups)

    def ExportSkillDictList(self):
        file_path = "skills.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Search Keyword","Resource Path","Groups","Prerequisites"])
            for key, value in self.skill_dict_list.items():
                name=key
                search = value.keyword_search
                path = value.resource_path
                groups =""
            
                for g in value.group_set:
                    groups += "["
                    groups += g
                    groups +="]"
                pre = ""
                for p in value.prerequisite:
                    pre += "["
                    pre += p
                    pre +="]"
                writer.writerow([name,search,path,groups,pre])
            file.close()

    def ExportGroupDictList(self):
        file_path = "groups.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Search Keyword","Resource Path","skills"])
            for key, value in self.group_dict_list.items():
                name=key
                search = value.keyword_search
                path = value.resource_path
                skills =""
                for s in value.skill_set:
                    skills += "["
                    skills += s
                    skills +="]"
                writer.writerow([name,search,path,skills])
            file.close()

    def ExportRolesDictList(self):
        file_path = "roles.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Search Keyword","Resource Path","Skills","Groups"])
            for key, value in self.role_dict_list.items():
                name=key
                search = value.keyword_search
                path = value.resource_path
                skills =""
                for s in value.skill_set:
                    skills += "["
                    skills += s
                    skills +="]"
                groups =""
                for g in value.group_set:
                    groups += "["
                    groups += g
                    groups +="]"
                writer.writerow([name,search,path,skills,groups])
            file.close()

    def ExportAliasDictList(self):
        file_path = "alias.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Word", "Also Known as"])
            for key, value in self.alias_dict_list.items():
                writer.writerow([key,value])
            file.close()

    def ExportIgnoreSet(self):
        with open('ignore.txt', 'w') as f:
            for i in  self.ignore_set:
                f.write(i)
                f.write("\n")
            f.close()

    def ExportNotFoundSet(self):
        with open('not found.txt', 'w') as f:
            for i in  self.not_found_set:
                f.write(i)
                f.write("\n")
            f.close()



    def LearningDocumentPrepare(self,company,skills,resume_skills):
        
        
        document = ""
        #new_list = [x for x in job_skills if x not in resume_skills]

        #check leetcode required
    

        # get all the learning resource reference link
        for s in skills:
            key = s
            if s in self.alias_dict_list:
                key = self.alias_dict_list.get(s)
                
            if key in self.skill_dict_list:
                # do something
                continue
            elif key in self.group_dict_list:
                # do something
                continue
            else:
                self.not_found_set.add(key)

        self.ExportNotFoundSet()
        
         




In [4]:
test = TechStack()

# top programming list
name_list = []
name_list.append("javascript")
name_list.append("typescript")
name_list.append("html")
name_list.append("css")
name_list.append("python")
name_list.append("sql")
name_list.append("bash")
name_list.append("java")
name_list.append("c++")
name_list.append("c")
name_list.append("php")
name_list.append("rust")
name_list.append("kotlin")
name_list.append("ruby")
name_list.append("lua")
name_list.append("dart")
name_list.append("assembly")
name_list.append("swift")
name_list.append("r")
name_list.append("matlab")
name_list.append("groovy")
name_list.append("delphi")
name_list.append("scala")
name_list.append("perl")
name_list.append("elixir")
name_list.append("objective c")
name_list.append("haskell")
name_list.append("gdscript")
name_list.append("lisp")
name_list.append("solidity")
name_list.append("clojure")
name_list.append("julia")
name_list.append("erlang")
name_list.append("fortran")
name_list.append("prolog")
name_list.append("zig")
name_list.append("ada")
name_list.append("ocaml")
name_list.append("sas")
name_list.append("crystal")
name_list.append("apl")
name_list.append("flow")
name_list.append("raku")
name_list.append("racket")
name_list.append("xhtml")
name_list.append("d")
name_list.append("scratch")
for i in range(len(name_list)):
    keyword = name_list[i] + " programming"
    test.AddSkillDictList(name_list[i], "programming", keyword,{"programming"})

name = "socket"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","network"} )
name = "rpc"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","network"} )
name = "checksum"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )
name = "multiprocessing"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","concurrency","parallelism"} )
name = "multithreading"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","concurrency","parallelism"} )
name = "asynchronous"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","concurrency","parallelism"} )
name = "serialization"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )
name = "deserialization"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )
name = "file io"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )
name = "encrypt"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","security"} )
name = "decrypt"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","security"} )
name = "requests"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","network","web"} )
name = "lru cache"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )


# microsoft
name = "windows"
test.AddSkillDictList(name, "microsoft", name , {"microsoft","os"} )
name = "windows server"
test.AddSkillDictList(name, "microsoft", name , {"microsoft","os","server"} )
name = "mircosoft sql"
test.AddSkillDictList(name, "microsoft", name , {"microsoft","database","data warehouse","sql"})
name = "mircosoft word"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","microsoft office"} )
name = "mircosoft excel"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","microsoft office"} )
name = "mircosoft powerpoint"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","microsoft office"} )
name = "mircosoft sharepoint"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","microsoft office"} )
name = "microsoft dynamics"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","microsoft office"} )
name = "mircosoft exchange"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","microsoft office"} )
name = "mircosoft visual studio"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","ide"} )
name = "mircosoft visual code"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","ide"} )
name= "power bi"
test.AddSkillDictList(name, "microsoft", name ,{"visualization","microsoft"} )
name = "c#"
test.AddSkillDictList(name, "programming", name, {"microsoft","programming"} )
name = "f#"
test.AddSkillDictList(name, "programming", name, {"microsoft","programming"} )
name = "powershell"
test.AddSkillDictList(name, "programming", name, {"microsoft","programming"} )
name = "visual basic"
test.AddSkillDictList(name, "programming", name, {"microsoft","programming" })
name = "visual basic net"
test.AddSkillDictList(name, "programming", name, {"microsoft","programming" })
name = "vba"
test.AddSkillDictList(name, "programming", name, {"microsoft","programming" })
name = "aspnet mvc"
test.AddSkillDictList(name, "web", name, {"microsoft","framework" })
name = "aspnet"
test.AddSkillDictList(name, "web", name, {"microsoft","framework" })
name = "aspnet core"
test.AddSkillDictList(name, "web", name, {"microsoft","framework"} )
name = "net"
test.AddSkillDictList(name, "web",  ".net", {"microsoft","framework" })
name = "net core"
test.AddSkillDictList(name, "web", ".net core", {"microsoft","framework" })
name = "wpf"
test.AddSkillDictList(name, "web", name, {"microsoft","framework" })
name = "linq"
test.AddSkillDictList(name, "web", name, {"microsoft","framework" })
name = "directx"
test.AddSkillDictList(name, "game", name, {"microsoft","framework","game" })
name = "microsoft game developer kit"
test.AddSkillDictList(name, "game", name, {"microsoft","framework","game" })
name = "microsoft quantum"
test.AddSkillDictList(name, "quantum", name , {"microsoft","quantum"})
#azure
name = "cosmosdb"
test.AddSkillDictList(name, "microsoft", "azure " + name, {"microsoft","cloud","azure","database","nosql"})
name = "azure sql database"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure","database","sql"})
name = "azure blobs"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure","storage"})
name = "azure file"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure"})
name = "azure elastic san"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
name = "azure tables"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
name = "azure queues"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure"})
name = "azure managed disks"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
name = "azure container storage"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
name = "azure container"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure","container"})
name = "azure data lake"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
name = "service bus"
test.AddSkillDictList(name, "microsoft", "azure " + name, {"microsoft","cloud","azure","message queue"})
name = "hdinsight"
test.AddSkillDictList(name, "microsoft", "azure " + name, {"microsoft","cloud","azure","message queue"})
name = "azure synapse analytics"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure","data warehouse","big data"})
name = "azure ai"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure","artificial intelligence"})
name = "azure montior"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure","monitoring"})
name = "azure pipelines"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure","continuous integration and continuous deployment"})
name = "arm"
test.AddSkillDictList(name, "microsoft", "azure " + name, {"microsoft","cloud","azure"})
name = "azure virtual machines"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure", "virtual machine"})

#google
name= "android"
test.AddSkillDictList(name, "google", name,{"google","os", "mobile"})
name= "go"
test.AddSkillDictList(name, "programming", name + " programming",{"google","programming"})
name= "android studio"
test.AddSkillDictList(name, "mobile", name,{"google","android", "mobile"})
name= "android sdk"
test.AddSkillDictList(name, "mobile", name,{"google","android", "mobile"})
name= "google docs"
test.AddSkillDictList(name, "google", name,{"google"})
name= "google sheets"
test.AddSkillDictList(name, "google", name,{"google"})
name= "google slides"
test.AddSkillDictList(name, "google", name,{"google"})
#google cloud
name = "cloud sql"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","sql"})
name = "alloydb"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","sql"})
name = "spanner"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","sql"})
name = "bigtable"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","key value"})
name = "firestore"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","document database"})
name = "firebase realtime database"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","document database"})
name = "memorystore"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","in memory database"})
name = "mongodb atlas"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","nosql"})
name = "bigquery"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","data warehouse","big data"})
name = "dataproc"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud"})
name = "google cloud storage"
test.AddSkillDictList(name, "google", name, {"google","cloud","google cloud","storage"})
name = "vertex ai"
test.AddSkillDictList(name, "google", name, {"google","cloud","google cloud","artificial intelligence"})
name = "cloud tasks"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","message queue"})
name = "pub sub"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","message queue"})
name = "google kubernetes engine"
test.AddSkillDictList(name, "google",  name, {"google","cloud","google cloud","container"})
name = "google cloud monitoring"
test.AddSkillDictList(name, "google", name, {"google","cloud","google cloud","monitoring"})
name = "cloud build"
test.AddSkillDictList(name, "google", "google "+ name, {"google","cloud","google cloud","continuous integration and continuous deployment"})
name = "google compute engine"
test.AddSkillDictList(name, "google",  name, {"google","cloud","google cloud","virtual machine"})

#aws
name = "s3"
test.AddSkillDictList(name, "amazon", "aws " + name, {"cloud","aws","storage"})
name = "amazon fsx"
test.AddSkillDictList(name, "amazon", name, {"cloud","aws"})
name = "amazon ebs"
test.AddSkillDictList(name, "amazon", name, {"cloud","aws","storage"})
name = "amazon file cache"
test.AddSkillDictList(name, "amazon", name, {"cloud","aws"})
name = "amazon drs"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws"})
name = "amazon neptune"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","database","graph database"})
name = "amazon rds"
test.AddSkillDictList(name, "amazon", name, {"cloud","aws","database","sql"})
name = "amazon aurora"
test.AddSkillDictList(name, "amazon", name, {"cloud","aws","database","sql"})
name = "amazon dynamodb"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","database","nosql"})
name = "amazon redshift"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","data warehouse"})
name = "amazon elasticache"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","data store"})
name = "amazon memorydb"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","database","in memory database"})
name = "amazon documentdb"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","database","document databaset"})
name = "amazon timestream"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","database","time series"})
name = "amazon  glue"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws"})
name = "amazon athena"
test.AddSkillDictList(name, "amazon", name, {"cloud","aws","big data"})
name = "amazon emr"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","big data"})
name = "amazon sns"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","message queue","big data"})
name = "amazon lake formation"
test.AddSkillDictList(name, "amazon", name, {"cloud","aws"})
name = "amazon sagemaker"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","artificial intelligence"})
name = "amazon ecs"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","container"})
name = "amazon fargate"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","container"})
name = "amazon lambda"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws"})
name = "amazon vpc"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws"})
name = "amazon cloudwatch"
test.AddSkillDictList(name, "amazon", name, {"cloud","aws","monitoring"})
name = "amazon sqs"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","message queue"})
name = "amazon codepipeline"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws","continuous integration and continuous deployment"})
name = "amazon ec2"
test.AddSkillDictList(name, "amazon",  name, {"cloud","aws", "virtual machine"})

# apple
name = "mac"
test.AddSkillDictList(name, "apple", name + " technology", {"apple","os"} )
name = "ios"
test.AddSkillDictList(name, "apple", name + " technology", {"apple","os","mobile"} )
name = "swift"
test.AddSkillDictList(name, "programming", name + " programming",{"apple","programming"})
name = "ios sdk"
test.AddSkillDictList(name, "apple", name ,{"apple","framework", "mobile","ios"})
name = "xcode"
test.AddSkillDictList(name, "apple", name ,{"apple","framework"})
# oracle
name = "oracle sql"
test.AddSkillDictList(name, "oracle", name, {"oracle","database"})
name = "pl-sql"
test.AddSkillDictList(name, "programming", name + " programming", {"oracle","database","programming"})
name= "oracle adf"
test.AddSkillDictList(name, "oracle", name + " framework",{"oracle"}  )
name = "oracle cloud"
test.AddSkillDictList(name, "oracle", name ,{"oracle","cloud"}  )
name = "oracle erp"
test.AddSkillDictList(name, "oracle", name ,{"oracle","cloud"}  )
name = "oracle virtual box"
test.AddSkillDictList(name, "oracle", name ,{"oracle","cloud", "virtual machine"}  )
name = "oracle c"
test.AddSkillDictList(name, "oracle", name, {"oracle"})
name = "oracle ebs"
test.AddSkillDictList(name, "oracle", name ,{"oracle","cloud"}  )
name = "oracle soa"
test.AddSkillDictList(name, "oracle", name ,{"oracle","cloud"}  )
name = "oracle linux"
test.AddSkillDictList(name, "oracle", name ,{"oracle", "cloud"} )
name = "oracle net"
test.AddSkillDictList(name, "oracle", name ,{"oracle", "cloud"} )

#ibm
name = "ibm db"
test.AddSkillDictList(name, "ibm", name, {"database","ibm"})
name = "ibm server"
test.AddSkillDictList(name, "ibm", name, {"server","ibm"})
name = "ibm mq"
test.AddSkillDictList(name, "ibm", name, {"message queue","ibm"})
name = "ibm watson"
test.AddSkillDictList(name, "ibm", name + " framework", {"ibm","artificial intelligence"})
name = "qiskit"
test.AddSkillDictList(name, "ibm", name , {"ibm","quantum"})

# vmware
name = "vmware vsphere"
test.AddSkillDictList(name, "vmware", name, {"cloud","vmware"} )
name = "vmware vcenter"
test.AddSkillDictList(name, "vmware", name, {"cloud","vmware"} )
name = "vmware esxi"
test.AddSkillDictList(name, "vmware", name, {"cloud","vmware"} )
name = "vmware certified professional"
test.AddSkillDictList(name, "vmware", name, {"vmware"} )

#sap
name = "hana"
test.AddSkillDictList(name, "sap", name + " database", {"database","sap"})
name = "sap bi"
test.AddSkillDictList(name, "sap", name , {"visualization","sap"})
name = "sap bw"
test.AddSkillDictList(name, "sap", name , {"data warehouse","sap"})
name = "sap basis"
test.AddSkillDictList(name, "sap", name , {"sap"})
name = "sap abap"
test.AddSkillDictList(name, "sap", name , {"sap", "programming"})

# apache
name = "accumulo"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "activemq"
test.AddSkillDictList(name, "message queue", "apache " + name , {"apache","message queue"})
name = "airavata"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "airflow"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "allura"
test.AddSkillDictList(name, "version control", "apache " + name , {"apache","git"})
name = "ambari"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "ant"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "ivy"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "apisix"
test.AddSkillDictList(name, "microservices", "apache " + name , {"apache","microservices"})
name = "aries"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "arrow"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "avro"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "axis"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "rampart"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "beam"
test.AddSkillDictList(name, "big data", "apache " + name , {"apache","big data"})
name = "bloodhound"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","git"})
name = "brooklyn"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "calcite"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "camel"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "carbondata"
test.AddSkillDictList(name, "big data", "apache " + name , {"apache","big data"})
name = "cassandra"
test.AddSkillDictList(name, "database", "apache " + name , {"apache","database","nosql"})
name = "cayenne"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "celix"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "cloudstack"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "cacoon"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "commons"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "bcel"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "logging"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "ognl"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "cordova"
test.AddSkillDictList(name, "mobile", "apache " + name , {"apache"})
name = "couchdb"
test.AddSkillDictList(name, "database", "apache " + name , {"apache","database", "nosql","document database"})
name = "cxf"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "derby"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","rdbms"})
name = "jdo"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "cayenne"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "ldap"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "fortress"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "scimple"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "drill"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "druid"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "empire-db"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "felix"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "flex"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "flink"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "flume"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","log"})
name = "fluo"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "freemarker"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "geronimmo"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "groovy"
test.AddSkillDictList(name, "programming", "apache " + name , {"apache","programming"})
name = "gump"
test.AddSkillDictList(name, "continuous integration", "apache " + name , {"apache"})
name = "hadoop"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "hbase"
test.AddSkillDictList(name, "big data", "apache " + name , {"apache","database","big data"})
name = "helix"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "hive"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","data warehouse"})
name = "http server"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "iceberg"
test.AddSkillDictList(name, "big data", "apache " + name , {"apache","big data"})
name = "ignite"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "impala"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "iotdb"
test.AddSkillDictList(name, "database", "apache " + name , {"apache","database","time series"})
name = "jackrabbit"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "james"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "jmeter"
test.AddSkillDictList(name, "test", "apache " + name , {"apache","testing"})
name = "kafka"
test.AddSkillDictList(name, "message queue", "apache " + name , {"apache","message queue"})
name = "kudu"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "kylin"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "log4j"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","log"})
name = "log4net"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","log"})
name = "lucene core"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "solr"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "mahout"
test.AddSkillDictList(name, "artificial intelligence", "apache " + name , {"apache","machine learning","artificial intelligence"})
name = "maven"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "doxia"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "messos"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "ftp server"
test.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
name = "mina"
test.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
name = "myface"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "mynewt"
test.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
name = "netbeans"
test.AddSkillDictList(name, "ide", "apache " + name , {"apache","ide"})
name = "nifi"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "nutch"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","web crawler"})
name = "nuttx"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","os"})
name = "ofbix"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "oozie"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "openjpa"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "opennlp"
test.AddSkillDictList(name, "artificial intelligence", "apache " + name , {"apache","natural language processing","artificial intelligence"})
name = "openoffice"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "orc"
test.AddSkillDictList(name, "big data", "apache " + name , {"apache","big data"})
name = "parquet"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "pdfbox"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "mod perl"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "phoenix"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "pig"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "pinot"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "pivot"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "poi"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "xmlbeans"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "pr"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "qpid"
test.AddSkillDictList(name, "message queue", "apache " + name , {"apache","message queue"})
name = "rocketmq"
test.AddSkillDictList(name, "message queue", "apache " + name , {"apache","message queue"})
name = "roller"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "royale"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "samza"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "shiro"
test.AddSkillDictList(name, "security", "apache " + name , {"apache","security"})
name = "singa"
test.AddSkillDictList(name, "artificial intelligence", "apache " + name , {"apache","artificial intelligence"})
name = "sling"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "solr"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "spamassassin"
test.AddSkillDictList(name, "security", "apache " + name , {"apache","security"})
name = "spark"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "storm"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "struts"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "subversion"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","git"})
name = "superset"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "synapse"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "systemds"
test.AddSkillDictList(name, "artificial intelligence", "apache " + name , {"apache","machine learning","artificial intelligence"})
name = "tapestry"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "tcl"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "tika"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "tinkerpop"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "tomcat"
test.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
name = "tomee"
test.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
name = "traffic server"
test.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
name = "turbine"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "velocity"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "axiom"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "wicket"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "xalan"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "xerces"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "batik"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "fop"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "yetus"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "zookeeper"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "hdfs"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})

# cloud
name = "suse"
test.AddSkillDictList(name, "cloud", name + " cloud",{"cloud"} )
name = "snowflake"
test.AddSkillDictList(name, "cloud", name, {"cloud","data warehouse"} )
name = "databricks"
test.AddSkillDictList(name, "cloud", name, {"cloud","data warehouse"} )
name = "citrix"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "salesforce"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "servicenow"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "akamai"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "talend"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "splunk"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "teradata"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "qualys"
test.AddSkillDictList(name, "security", name, {"cloud","security"} )
name = "heroku"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "alibaba cloud"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "cloudera"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
#database
name = "mysql"
test.AddSkillDictList(name, "database", name + " database", {"database","sql"})
name = "postgresql"
test.AddSkillDictList(name, "database", name + " database", {"database","sql"})
name = "mongodb"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql","document database"})
name = "cassandra"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql","distributed"})
name = "redis"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql","in memory database"})
name = "neo4j"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql","graph database"})
name = "sqlite"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "mariadb"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "elasticsearch"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "couchbase"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "orientdb"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql"})
name = "firebirdsql"
test.AddSkillDictList(name, "database", name + " database", {"database","sql"})
name = "influxdb"
test.AddSkillDictList(name, "database", name + " database", {"database","time series"})
name = "kdb"
test.AddSkillDictList(name, "database", name + " database", {"database","time series"})
name = "prometheus"
test.AddSkillDictList(name, "database", name + " database", {"database","time series"})
name = "chroma"
test.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
name = "pinecone"
test.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
name = "faiss"
test.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
name = "tibero"
test.AddSkillDictList(name, "database", name + " database", {"database"})


#container
name = "docker"
test.AddSkillDictList(name, "container", name + " container",{"container"} )
name = "lxc"
test.AddSkillDictList(name, "container", name + " container",{"container"} )
name = "container linux"
test.AddSkillDictList(name, "container", name ,{"container"} )
name = "portainer"
test.AddSkillDictList(name, "container", name + " container",{"container"} )
name = "messos"
test.AddSkillDictList(name, "container", name + " container",{"container"} )
name = "kubernetes"
test.AddSkillDictList(name, "container", name + " container",{"container"} )

#os

name = "linux"
test.AddSkillDictList(name, "os", name + " technology", {"os"} )
name = "solaris"
test.AddSkillDictList(name, "os", name + " technology", {"microsoft","os"} )
name = "vxworks"
test.AddSkillDictList(name, "os", name + " technology", {"os"} )

# visualization
# monitoring tool
name = "datadog"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "grafana"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "nagios"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "micro focus sitescope"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "sensu"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "prometheus"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "observium"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "checkmk"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "new relic"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "dynatrace"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "atera"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "paessler prtg"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "zabbix"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "appdynamics"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "logrocket"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name = "logz.io"
test.AddSkillDictList(name, "visualization", name,{"visualization"} )
name= "kibana"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "tableau"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "qlikview"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "qliksense"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )

name = "glassfish"
test.AddSkillDictList(name, "network", name + " application servers",{"network"} )
name = "jetty"
test.AddSkillDictList(name, "network", name + " application servers",{"network"} )
name = "wildfly"
test.AddSkillDictList(name, "network", name + " application servers",{"network"} )


# top web framework
name_list.clear()
name_list.append("nodejs")
name_list.append("react")
name_list.append("jquery")
name_list.append("express")
name_list.append("angular")
name_list.append("nextjs")
name_list.append("asp.net.core")
name_list.append("vuejs")
name_list.append("wordpress")
name_list.append("aspnet")
name_list.append("flask")
name_list.append("spring boot")
name_list.append("django")
name_list.append("laravel")
name_list.append("fastapi")
name_list.append("angularjs")
name_list.append("svelte")
name_list.append("ruby on rails")
name_list.append("nestjs")
name_list.append("blazor")
name_list.append("nuxtjs")
name_list.append("symfony")
name_list.append("deno")
name_list.append("gatsby")
name_list.append("fastify")
name_list.append("phoenix")
name_list.append("drupal")
name_list.append("codelgniter")
name_list.append("solidjs")
name_list.append("remix")
name_list.append("elm")
name_list.append("play")
name_list.append("lit")
name_list.append("qwik")
name_list.append("reactjs")
for i in range(len(name_list)):
    keyword = name_list[i] + " framework/library"
    test.AddSkillDictList(name_list[i], "web", keyword, {"framework","web"})

# top distributed
name = "mapreduce"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name = "samza"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name = "flink"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name = "heron"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name = "kudu"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name = "presto"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name = "centos"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name= "pyspark"
test.AddSkillDictList(name, "distributed", name , {"distributed","framework"},{"python", "spark"}  )
# mq
name = "rabbitmq"
test.AddSkillDictList(name, "message queue", name , {"message queue"})
name = "ironmq"
test.AddSkillDictList(name, "message queue", name , {"message queue"})
name = "mulesoft"
test.AddSkillDictList(name, "message queue", name + " message queue" , {"message queue"})
name = "pubsub+"
test.AddSkillDictList(name, "message queue", name + " message queue" , {"message queue"})
name = "zeromq"
test.AddSkillDictList(name, "message queue", name + " message queue" , {"message queue"})
name = "tibco"
test.AddSkillDictList(name, "message queue", name + " message queue" , {"message queue"})

# top ai framework
name = "torch"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","natural language processing"})
name = "pytorch"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","natural language processing"})
name = "scikit-learn"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","natural language processing","visualization","data science"})
name = "tensorflow"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","computer vision","natural language processing"})
name = "keras"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","computer vision","natural language processing"})
name = "langchain"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
name = "cuda"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
name = "transformers"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
name = "opennn"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "openai"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
name = "pybrain"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "cntk"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "dl4j"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "theano"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "mxnet"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "caffe"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "xgboost"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
name = "opencv"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "viso suite"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "yolo"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "boofcv"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "openvino"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "simplecv"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "deepface"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "spacy"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
name = "xlm"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
name = "chainer"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
name = "gensim"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
name = "nltk"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
name = "matplotlib"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
name = "seaborn"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
name = "pandas"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
name = "numpy"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
name = "data modeling"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
name = "data mining"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
name = "language model"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science","natural language processing"})
# top microservices
name = "spring"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})
name = "spring boot"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})
name = "spring cloud"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})
name = "expressjs"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"node.js"})
name = "django"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"python"})
name = "django rest"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"python"})
name = "go kit"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"go"})
name = "micronaut"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java","groovy","kotlin"})
name = "fastapi"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"python"})
name = "helidon"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})
name = "lagom"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java","scala"})
name = "quarkus"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})

name= "xml"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "json"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "binary"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "xpath"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "query"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "ajax"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "csv"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "hexadecimal"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "xsl"
test.AddSkillDictList(name, "format", name + " format",{"format"} )

# infa code
name = "terraform"
test.AddSkillDictList(name, "infrastructure code", name, {"infrastructure code"} )
name = "spacelift"
test.AddSkillDictList(name, "infrastructure code", name, {"infrastructure code"} )
name = "ansible"
test.AddSkillDictList(name, "infrastructure code", name, {"infrastructure code"} )

# web crawler
name = "selenium"
test.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"})
name = "selenium base"
test.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"},{"selenium"})
name = "scrapy"
test.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"},{"python"})
name = "mechanicalsoup"
test.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"},{"python"})
name = "beautiful soup"
test.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"},{"python"})

#top frontend
name_list.clear()
name_list.append("react")
name_list.append("tailwind css")
name_list.append("angular")
name_list.append("vuejs")
name_list.append("bootstrap")
name_list.append("mdl")
name_list.append("bulma")
name_list.append("mdl")
name_list.append("semantic ui")
name_list.append("foundation")
name_list.append("ulkit")
name_list.append("reactjs")
for i in range(len(name_list)):
    keyword = name_list[i] + " frontend"
    test.AddSkillDictList(name_list[i], "frontend", keyword, {"frontend","fullstack"})

# top backend
name = "spring boot"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"java"})
name = "django"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"python"})
name = "expressjs"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"nodejs"})
name = "laravel"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"php"})
name = "ruby on rails"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"ruby"})
name = "asp.net core"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"c#"})
name = "nestjs"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"typescript","nodejs"})
name = "koajs"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"nodejs"})
name = "phoenix"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"elixir"})

# top mobile development framework
name_list.clear()
name_list.append("flutter")
name_list.append("react native")
name_list.append("swiftic")
name_list.append("kotlin")
name_list.append("ionic")
name_list.append("xamarin")
name_list.append("corona")
name_list.append("titanium")
name_list.append("nativescript")
name_list.append("jquery mobile")
for i in range(len(name_list)):
    keyword = name_list[i] + " mobile development framework"
    test.AddSkillDictList(name_list[i], "mobile", keyword, {"mobile"})

# top test automation frameworks
name = "selenium"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"java","python","c#"})
name = "appium"
test.AddSkillDictList(name, "testing", name + " framework", {"testing","mobile"})
name = "testng"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"java","junit"})
name = "junit"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"java"})
name = "cucumber"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"})
name = "testcomplete"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"javascript","python","visual basic"})
name = "robot"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"})
name = "cypress"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"javascript"})
name = "junit jupiter"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"java","junit"})
name = "pytest"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"python"})
name = "rspec"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"})
name = "jenkins"
test.AddSkillDictList(name, "testing", name + " framework", {"testing","devops"})
name = "galen"
test.AddSkillDictList(name, "testing", name + " framework", {"testing","web"})
name = "qtp"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"})
# top blockchain
name = "ethereum"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "ibm blockchain"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "hyperledger"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "eos"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "corda"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "ripple"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "quorum"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "iota"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "neo"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "tezos"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "stellar"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "sui network"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "hedera"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "aptos"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "flow"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "fantom"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "solana"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "tron"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "binance smart chain"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "icp"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})

# top game developer
name = "unreal engine"
test.AddSkillDictList(name, "game", name + " engine", {"game"},{"c++"})
name = "unity"
test.AddSkillDictList(name, "game", name + " engine", {"game"},{"c#"})
name = "godot"
test.AddSkillDictList(name, "game", name + " engine", {"game"},{"c#"})
name = "cocos2d"
test.AddSkillDictList(name, "game", name + " engine", {"game"})
name = "rpg maker"
test.AddSkillDictList(name, "game", name + " engine", {"game"})
name = "vulkan"
test.AddSkillDictList(name, "game", name + " graphics framework", {"game"})
name = "opengl"
test.AddSkillDictList(name, "game", name + " graphics framework", {"game"})
name = "physx"
test.AddSkillDictList(name, "game", name + " physics engine", {"game"})
name = "bullet"
test.AddSkillDictList(name, "game", name + " physics engine", {"game"})
name = "open dynamics engine"
test.AddSkillDictList(name, "game", name + " physics engine", {"game"})
name = "fmod"
test.AddSkillDictList(name, "game", name + " audio framework", {"game"})
name = "glm"
test.AddSkillDictList(name, "game", name + " math framework", {"game"})
name = "assimp"
test.AddSkillDictList(name, "game", name + " library", {"game"})
name = "shader"
test.AddSkillDictList(name, "game", name + " programming", {"game"})
name = "vertices"
test.AddSkillDictList(name, "game", name + " computer graphic", {"game"})
name = "indices"
test.AddSkillDictList(name, "game", name + " computer graphic", {"game"})
#quantum
name = "tensorflow quantum"
test.AddSkillDictList(name, "quantum", name , {"quantum"})
name = "cuquantum"
test.AddSkillDictList(name, "quantum", name , {"quantum"})

# top framework/libraey
name_list.clear()
name_list.append("flutter")
name_list.append("react native")
name_list.append("electron")
name_list.append("qt")
name_list.append("swiftui")
name_list.append("xamarin")
name_list.append("lonic")
name_list.append("gtk")
name_list.append("cordova")
name_list.append("net maui")
name_list.append("tauri")
name_list.append("capacitor")
name_list.append("tidyverse")
name_list.append("quarkus")
name_list.append("ktor")
name_list.append("mfc")
name_list.append("jax")
name_list.append("uno platform")
name_list.append("jdbc")
name_list.append("jpa")
name_list.append("sqlalchemy")
name_list.append("jsp")
name_list.append("jboss")
name_list.append("extjs")
for i in range(len(name_list)):
    keyword = name_list[i] + " framework/library"
    test.AddSkillDictList(name_list[i], "framework", keyword,{"framework"})

name = "fleet"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "intellij"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "pycharm"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "webstorm"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "phpstorm"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "rider"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "clion"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "datalore"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "datagrip"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "rubymine"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "goland"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "resharper c++"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "rustrover"
test.AddSkillDictList(name, "ide", name ,{"ide"})
name = "aqua"
test.AddSkillDictList(name, "ide", name ,{"ide"})
name = "anaconda"
test.AddSkillDictList(name, "ide", name ,{"ide"})

name = "gitlab"
test.AddSkillDictList(name, "git", name ,{"continuous integration and continuous deployment","git","devops"} )
name = "bitbucket"
test.AddSkillDictList(name, "git", name ,{"continuous integration and continuous deployment","git","devops"} )
name = "github"
test.AddSkillDictList(name, "git", name ,{"git","devops"} )
name = "preforce"
test.AddSkillDictList(name, "git", name ,{"git","devops"} )

# cl/cd

test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "buddy"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "teamcity"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "circleci"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "tracisci"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "semaphore"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "puppet"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "gradle"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )


# pen testing
name = "nmap"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "w3af"
test.AddSkillDictList(name, "security", name +" pen test" , {"security","penetration testing"})
name = "tenable nessus"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "onsecurity"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "wireshark"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "zap"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "nikto"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "cobalt"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "rapid7"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "astra security"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})


name ="jira"
test.AddSkillDictList(name, "documentation", name , {"documentation"} )
name ="trello"
test.AddSkillDictList(name, "documentation", name , {"documentation"} )
name ="confluence"
test.AddSkillDictList(name, "documentation", name , {"documentation"} )
name ="kanban"
test.AddSkillDictList(name, "documentation", name , {"documentation"} )

name = "esri"
test.AddSkillDictList(name, "software or tool", name + " technology",{"arcgis"}  )
name = "postman"
test.AddSkillDictList(name, "testing ", name,{"monitoring","testing","web"} )
name = "etl"
test.AddSkillDictList(name, "cloud", name  ,{"cloud"})
name = "control m"
test.AddSkillDictList(name, "software or tool", name)
name = "loadrunner"
test.AddSkillDictList(name, "software or tool", name)
name = "tibco ems"
test.AddSkillDictList(name, "software or tool", name)
name = "akka"
test.AddSkillDictList(name, "software or tool", name)
name = "logstash"
test.AddSkillDictList(name, "software or tool", name)
name = "ubuntu"
test.AddSkillDictList(name, "software or tool", name)

name = "ip address"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "mac address"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "firewall"
test.AddSkillDictList(name, "network", name ,{"network","security"} )
name = "domain name system"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "area network"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "storage area network"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "dhcp"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "network protocol"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "network topology"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "network devices"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "broadcasting"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "subnet"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "subnet mask"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "soap"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "ospf"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "access control"
test.AddSkillDictList(name, "security", name ,{"network","architecture","security"} )
name = "xmpp"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "eigrp"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "nfs"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "vrrp"
test.AddSkillDictList(name, "network", name ,{"network"} )

name = "array"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "list"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "tuple"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "stack"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "queue"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "tree"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "graph"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "trie"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "hash table"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )

name = "abstraction"
test.AddSkillDictList(name, "oop", name + " oop",{"oop","architecture"} )
name = "encapsulation"
test.AddSkillDictList(name, "oop", name + " oop",{"oop","architecture"} )
name = "inheritance"
test.AddSkillDictList(name, "oop", name + " oop",{"oop","architecture"} )
name = "polymorphism"
test.AddSkillDictList(name, "oop", name + " oop",{"oop","architecture"} )
name = "object"
test.AddSkillDictList(name, "oop", name + " oop",{"oop","architecture"} )
name = "class"
test.AddSkillDictList(name, "oop", name + " oop",{"oop","architecture"} )

name = "factory"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "builder"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "dependency injection"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "lazy"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "object pool"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "multition"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "raii"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "prototype"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "singleton"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "visitor"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "join"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "lock"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "mvc"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "actors"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "component"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "strategy"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "observer"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "adapter"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "state"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "creational"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "structural"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "behavioral"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "decorator"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "facade"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "mvvm"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )


name = "monolithic system"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"} )
name = "distributed system"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "structured analysis"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "structured design"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "system design strategy"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "database sharding"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "scaling"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "load balancer"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "caching"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "ooad"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "public cloud"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "private cloud"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "hybrid cloud"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "file storage"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "block storage"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "object storage"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "sql"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "nosql"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "availability"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})

name= "software development life cycle"
test.AddSkillDictList(name, "other", name ,{"architecture"})
name= "agile"
test.AddSkillDictList(name, "other", name ,{"architecture"})


skill_set ={"c++","java","python","c","c#","javascript","typescript","php","swift","kotlin","dart","go","ruby","scala","rust","racket","erlang","elixir"}
test.AddGroupDictList("leetcode",skill_set)




test.ExportSkillDictList()
test.ExportGroupDictList()

In [5]:
test.AddAliasDictList("microsoft net","net")
test.AddAliasDictList("microsoft aspnet","aspnet")
test.AddAliasDictList("microsoft windows","windows")
test.AddAliasDictList("exchange","mircosoft exchange")
test.AddAliasDictList("word","mircosoft word")
test.AddAliasDictList("excel","mircosoft excel")
test.AddAliasDictList("dynamics","mircosoft excel")
test.AddAliasDictList("powerpoint","mircosoft powerpoint")
test.AddAliasDictList("sharepoint","mircosoft sharepoint")
test.AddAliasDictList("microsoft sharepoint server","mircosoft sharepoint")
test.AddAliasDictList("microsoft azure","azure")
test.AddAliasDictList("microsoft sql server","microsoft sql")
test.AddAliasDictList("sap hana","hana")
test.AddAliasDictList("redshift","amazon redshift")
test.AddAliasDictList("amazon web","aws")
test.AddAliasDictList("amazon cloud","aws")
test.AddAliasDictList("aurora","amazon aurora")
test.AddAliasDictList("rds","amazon rds")
test.AddAliasDictList("core java","java")
test.AddAliasDictList("java","javascript")
test.AddAliasDictList("vb script","visual basic")
test.AddAliasDictList("gcp","google cloud")
test.AddAliasDictList("cloud computing","cloud")
test.AddAliasDictList("microsoft net","net")
test.AddAliasDictList("restful","requests")
test.AddAliasDictList("selenium webdriver","selenium")
test.AddAliasDictList("ocr","computer vision")
test.AddAliasDictList("data visualisation","visualisation")
test.AddAliasDictList("powerbi","power bi")
test.AddAliasDictList("vbnet","visual basic net")
test.AddAliasDictList("tdd","testing")
test.AddAliasDictList("scrum","agile")
test.AddAliasDictList("sql query","sql")
test.AddAliasDictList("wlan","area network")
test.AddAliasDictList("vlan","area network")
test.AddAliasDictList("wan","area network")
test.AddAliasDictList("aspnet web","aspnet")
test.AddAliasDictList("svn","subversion")
test.AddAliasDictList("vcp","vmware certified professional")
test.AddAliasDictList("uat","testing")
test.AddAliasDictList("webphere mq","ibm mq")
test.AddAliasDictList("snmp","network protocol")
test.AddAliasDictList("http","network protocol")
test.AddAliasDictList("protocol","network protocol")
test.AddAliasDictList("sql server","mircosoft sql")
test.AddAliasDictList("continuous integration","continuous integration and continuous delivery")
test.AddAliasDictList("continuous delivery","continuous integration and continuous delivery")
test.AddAliasDictList("ci","continuous integration and continuous delivery")
test.AddAliasDictList("cd","continuous integration and continuous delivery")
test.AddAliasDictList("sdlc","software development life cycle")
test.AddAliasDictList("version control","git")
test.AddAliasDictList("windows powershell","powershell")
test.AddAliasDictList("cloudwatch","amazon cloudwatch")
test.AddAliasDictList("dynamodb","amazon dynamodb")
test.AddAliasDictList("ecs","amazon ecs")
test.AddAliasDictList("ec2","amazon ec2")
test.AddAliasDictList("aws lambda","amazon lambda")
test.AddAliasDictList("shaders","shader")
test.AddAliasDictList("trading system","system design")
test.AddAliasDictList("payment system","system design")
test.AddAliasDictList("cryptography","encrpyt")
test.ExportAliasDictList()




In [6]:
f = open("sample skill.txt", "r")
temp = set()
for c in f:
    c = c.replace("\n", "")
    c = c.replace("/", "-")
    # keep +, #, - only
    c = c.replace(".", "")
    c = c.replace(",", "")
    c = c.lower()
    # remove anything ()
    if c.find('(') != -1:
        c = c.split("(")[0]
        c = c.rsplit()[0]


    c = c.replace("apache ","")
    c = c.replace(" programming","")
    c = c.replace(" scripting","")    
    c = c.replace(" development","")    
    c = c.replace(" application","")   
    c = c.replace(" framework","")
    c = c.replace(" api","")
    c = c.replace(" platforms","") 
    c = c.replace(" platform","") 
    c = c.replace(" services","") 
    c = c.replace(" service","") 
   
    c = c.replace("webs","web") 
    c = c.replace("website","web") 
    c = c.replace(" tools","") 
    c = c.replace(" tool","")
    c = c.replace("atlassian ","")

    if c.find('ms ') !=-1:
         c = c.replace("ms","microsoft") 
    
    
    if c.find('security') !=-1:
        temp.add("security")
    elif c.find('test') !=-1 and c.find('penetration') ==-1:
        temp.add("testing")
    elif c.find('architecture') !=-1:
        temp.add("architecture")
    elif c.find('network') !=-1:
        temp.add("network")
    elif c.find('snowflake') !=-1:
        temp.add("snowflake")
    elif c.find('cloud ') !=-1:
        temp.add("cloud")
    elif c.find('management') !=-1:
        continue
    elif c.find('customer') !=-1:
        continue
    elif c.find('business') !=-1:
        continue
    elif c.find('sales') !=-1:
        continue
    elif c.find('support') !=-1:
        continue
    else:
        temp.add(c)
f.close
test.LearningDocumentPrepare(None, temp, None)

In [7]:

data_set = ""
f = open("not found.txt", "r")
for c in f:
    c = c.replace("\n", "")
    data_set += c
    data_set += " "
f.close()
split_it = data_set.split() 
counter = {}
for w in split_it:
    if w not in counter:
        counter[w] = 0
    counter[w] +=1
my_keys = sorted(counter, key=counter.get, reverse=True)[:100]
print(my_keys)

['design', 'data', 'analysis', 'skills', 'it', 'technical', 'technology', 'engineering', 'systems', 'and', 'microsoft', 'to', 'oracle', 'software', 'project', 'sap', 'system', 'user', 'information', 'planning', 'industry', 'financial', 'adobe', 'administration', 'team', 'server', 'windows', 'strategy', 'of', 'product', 'work', 'web', 'risk', 'integration', 'communication', 'infrastructure', 'leadership', 'research', 'database', 'quality', 'visual', 'operations', 'process', 'digital', 'service', 'mobile', 'audit', 'linux', 'writing', 'communications', 'banking', 'office', 'day', 'analytical', 'training', 'problem', 'marketing', 'language', 'unix', 'migration', 'requirements', 'video', 'public', 'ict', 'implementation', 'operating', 'online', 'analytics', 'corporate', 'delivery', 'iso', 'desktop', 'sql', 'erp', 'intelligence', 'healthcare', 'solving', 'control', 'performance', 'servers', 'ibm', 'pressure', 'assessment', 'computer', 'safety', 'compliance', 'maintenance', 'ability', 'confi